In [1]:
import os
os.environ['OPENAI_API_KEY']=''

In [10]:
import logging
import os
import sys
sys.path.append(os.getcwd() + '/src') # Add the 'src' directory to the system path to allow imports
from config import llm_config_35, log_config, vessel_msg
from request_classifier import RequestClassifierAgent
from chat import ChatAgent
from vessel_only import VesselAgent

# Configure logging as per the configuration in config.py
logging.basicConfig(**log_config)

# Initialize the Request Classifier Agent
request_classifier = RequestClassifierAgent(llm_config=llm_config_35)

# Initialize the Chat Agent
chat_agent = ChatAgent(llm_config=llm_config_35, system_message="Welcome to Digital Assistant for Digital Twin Earth!")

#Initialize the Vessel Agent
vessel_agent = VesselAgent(llm_config_35)

def main():
    messages = ['DA4DTE: I am DA4DTE. How can I help you?']
    # Simulating user input
    user_input = 'hi'
    while user_input!='bye':
        answer = ""
        user_input = input(messages[-1]+"\n(say 'bye' to finish): ")
        messages.append('USER: '+user_input)

        # Check if the input contains an image context or another specific context
        contains_image = False

        # Request Classifier Agent tries to classify the request
        request_existence, detected_category = request_classifier.request_existence_and_classification(user_input, contains_image=contains_image)

        if not request_existence or detected_category == 'None':
            # If no specific request or category detected, activate Chat Agent
            final_answer = chat_agent.generate_reply([{"content":user_input, "role": "user"}])
        else:
            a = detected_category
            #SEARCH BY METADATA -> ONLY FOR VESSELS
            if detected_category == 'IMAGE_RETRIEVAL_BY_CAPTION':
                is_vessel, certainty = vessel_agent.analyze_vessel_topic(user_input)
            # If a specific request is classified, show the engine's answer with a placeholder for link
                if not is_vessel:
                    answer = vessel_msg 

            # IF VESSELS -> SEARCH BY METADATA
            elif detected_category == 'IMAGE_RETRIEVAL_BY_METADATA':
                is_vessel, certainty = vessel_agent.analyze_vessel_topic(user_input)
                if is_vessel:
                    a = 'IMAGE_RETRIEVAL_BY_CAPTION'
            detected_category = a

            if answer !="":
                final_answer=answer
            else:
                final_answer = f""" {detected_category} answer [Link]"""
        messages.append('DA4DTE: '+final_answer)

    return messages

if __name__ == "__main__":
    dialog = main()


user (to guidance_labeler):


            Possible request types are: IMAGE_RETRIEVAL_BY_CAPTION, IMAGE_RETRIEVAL_BY_METADATA, GEOGRAPHY_QA, None.
            Label the USER_INPUT via the following instructions:

            The label must be a JSON of the format:
{
    "request_existence": bool,
    "explanation": str,
    "request_category": str
}

            USER_INPUT: Give me an image with vessels.
        

--------------------------------------------------------------------------------
guidance_labeler (to user):

{
    "request_existence": true,
    "explanation": "The user is requesting an image with vessels.",
    "request_category": "IMAGE_RETRIEVAL_BY_CAPTION"
}

--------------------------------------------------------------------------------
user (to vessel_agent):


            Determine whether the input relates to the vessel domain, including stating the absence of them.
            Follow these labeling instructions:
            
The label must be a JSON of the format